In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, month, year, dayofweek

In [5]:
spark = SparkSession.builder \
    .appName("ALAZ_ML_Phase") \
    .master("local[3]") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

25/12/14 02:18:27 WARN Utils: Your hostname, turan-H110 resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface enp2s0)
25/12/14 02:18:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/14 02:18:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/14 02:18:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/14 02:18:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [6]:

# Veriyi oku
df = spark.read.parquet("../data/processed/ALAZ_Final_Dataset_Cleaned.parquet")

# Şemayı yazdır (Veri tipleri doğru mu?)
df.printSchema()

# Toplam satır sayısını tekrar teyit edelim
print(f"Net Veri Seti Boyutu: {df.count()}")

root
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- tarih: date (nullable = true)
 |-- saat: integer (nullable = true)
 |-- yukselti: double (nullable = true)
 |-- egim: double (nullable = true)
 |-- baki: double (nullable = true)
 |-- sicaklik: double (nullable = true)
 |-- ruzgar_hizi: double (nullable = true)
 |-- bagil_nem: double (nullable = true)
 |-- yagis: double (nullable = true)
 |-- toprak_sicakligi: double (nullable = true)
 |-- NDVI: double (nullable = true)
 |-- label: integer (nullable = true)

Net Veri Seti Boyutu: 370100


In [8]:
df.select("yukselti", "egim", "sicaklik", "ruzgar_hizi", "bagil_nem", "NDVI", "label").describe().show(vertical=True)

-RECORD 0---------------------------
 summary     | count                
 yukselti    | 370100               
 egim        | 370100               
 sicaklik    | 370100               
 ruzgar_hizi | 370100               
 bagil_nem   | 370100               
 NDVI        | 370100               
 label       | 370100               
-RECORD 1---------------------------
 summary     | mean                 
 yukselti    | 842.3759065117536    
 egim        | 3.7088489597406107   
 sicaklik    | 19.047093796364894   
 ruzgar_hizi | 2.2050384961846823   
 bagil_nem   | 47.17273755025109    
 NDVI        | 0.30727785220210474  
 label       | 0.5419238043771953   
-RECORD 2---------------------------
 summary     | stddev               
 yukselti    | 606.2057477598167    
 egim        | 5.036224193697473    
 sicaklik    | 11.420272841872887   
 ruzgar_hizi | 1.3968531721176776   
 bagil_nem   | 24.994519551735404   
 NDVI        | 0.177269840028533    
 label       | 0.4982399676586143   
-

In [9]:
print("Yangın Durumu Dağılımı (1: Var, 0: Yok)")
df.groupBy("label").count().show()

Yangın Durumu Dağılımı (1: Var, 0: Yok)
+-----+------+
|label| count|
+-----+------+
|    1|200566|
|    0|169534|
+-----+------+



In [10]:
from pyspark.sql.functions import col

print("Yangınların Saate Göre Dağılımı:")
df.filter(col("label") == 1) \
  .groupBy("saat") \
  .count() \
  .orderBy("saat") \
  .show(24)

Yangınların Saate Göre Dağılımı:
+----+-----+
|saat|count|
+----+-----+
|   0| 1517|
|   1|  143|
|   6|  116|
|   7| 8825|
|   8|53770|
|   9|17315|
|  10|52237|
|  11|30561|
|  12| 1255|
|  13|   49|
|  17|  136|
|  18| 5485|
|  19|19143|
|  20| 3887|
|  21|    1|
|  22|  878|
|  23| 5248|
+----+-----+



In [11]:
from pyspark.sql.functions import month

print("Yangınların Aylara Göre Dağılımı:")
df.filter(col("label") == 1) \
  .withColumn("ay", month(col("tarih"))) \
  .groupBy("ay") \
  .count() \
  .orderBy("ay") \
  .show()

Yangınların Aylara Göre Dağılımı:
+---+-----+
| ay|count|
+---+-----+
|  1| 3100|
|  2| 2487|
|  3| 3722|
|  4| 2725|
|  5| 4336|
|  6|35534|
|  7|22903|
|  8|30939|
|  9|40609|
| 10|29854|
| 11|18618|
| 12| 5739|
+---+-----+

